In [2]:
from langchain.indexes import VectorstoreIndexCreator
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import TextLoader
import textwrap

In [3]:

#url = "https://diabetes.org/tools-support/diabetes-prevention/diabetes-myths"  # Replace with the URL you want to extract text from
def print_wrapped_text(text, line_width, answer_file):
    wrapped_text = textwrap.fill(text, width=line_width)
    # Open the file in append mode
    with open(answer_file, 'a') as file:
    # Append the string to the file
        file.write("\n"+wrapped_text+"\n"+"-"*100)

def parse_document(url, category):

    # Send a GET request to the URL
    response = requests.get(url)
    file_name = category + "_content.txt"
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the response using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the text from the parsed HTML
        text_content = soup.get_text()
        # Save the text content to a file
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(text_content)

        print(f"Text content was saved to {file_name}")
        loader = TextLoader(file_name)
        index = VectorstoreIndexCreator().from_loaders([loader])
        qs = index.query("Please list all the questions I can ask")
        questions = qs.splitlines()
        queries = [line.lstrip("- ") for line in questions[1:]]
        question = "What questions I can ask based on the content? List each question in one line please."
        a = index.query(question)
        print(a)
        questions = a.splitlines()
        # Remove empty items from the list
        questions = [line.strip() for line in questions if line.strip()]
        print(questions)
        count = 1
        answer_file = category+"_answer.txt"
        print(f"Answers were saved to {answer_file}")
        for question in questions:  
            print_wrapped_text(f"Question {str(count)} - {question}", 100, answer_file)
            a = index.query(question)
            print_wrapped_text(f"Answer - {a}", 100, answer_file)
    
            count += 1
    else:
        # Handle the case when the request was not successful
        print("Error:", response.status_code)
    

In [4]:
url = "https://diabetes.org/healthy-living/recipes-nutrition"
category = "nutrition"
parse_document(url, category)

Text content was saved to nutrition_content.txt

What are the guidelines for finding an eating pattern that works for me?
What information can I find on food labels?
What tools can I use to know my risk for diabetes?
What lifestyle changes can I make to help manage my diabetes?
What are the symptoms of Type 1 and Type 2 diabetes?
What are the treatments and care for Type 1 and Type 2 diabetes?
What are the risks of diabetes for cardiovascular disease, chronic kidney disease, and stroke?
What resources are available for people with diabetes?
What is the Diabetes Plate Method?
What are the nutrients I should be aware of?
What is the Health Equity Now initiative?
['What are the guidelines for finding an eating pattern that works for me?', 'What information can I find on food labels?', 'What tools can I use to know my risk for diabetes?', 'What lifestyle changes can I make to help manage my diabetes?', 'What are the symptoms of Type 1 and Type 2 diabetes?', 'What are the treatments and car